In [1]:
# Imports
import os 
import pandas as pd
from django.core.exceptions import ValidationError
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from datetime import date


In [ ]:
##Maternal Window
from edc_appointment.models import Appointment

pids = []

for a in OnScheduleCohortABirth.objects.all():
    try:
        md = MaternalDelivery.objects.get(subject_identifier=a.subject_identifier,
                                          )
    except MaternalDelivery.DoesNotExist:
        pass
    else:
        if md.delivery_datetime.date() < a.onschedule_datetime.date():
            pids.append(a.subject_identifier)



In [ ]:
pids

In [ ]:
for pid in pids:

    subject_identifier = pid

    print(">>>>>>>>>>>>>", pid)

    ons = OnScheduleCohortABirth.objects.get(subject_identifier=subject_identifier)

    ons.schedule_name = ons.schedule_name + 'x'
    ons.save()

    ap0 = Appointment.objects.get(subject_identifier=subject_identifier,
                                  schedule_name='a_birth1_schedule1',
                                  visit_code_sequence=0)

    ap0.schedule_name = ons.schedule_name
    ap0.save()

    # resave child birth

    mdv = MaternalDelivery.objects.get(subject_identifier=subject_identifier)

    mdv.save()

    ap1 = Appointment.objects.get(subject_identifier=subject_identifier,
                                  schedule_name='a_birth1_schedule1',
                                  visit_code_sequence=0)

    ap1.user_modified = ap0.user_modified
    ap1.appt_status = ap0.appt_status
    ap1.appt_reason = ap0.appt_reason
    ap1.appt_datetime = ap0.appt_datetime

    ap1.save()

    try:
        v1 = MaternalVisit.objects.get(appointment=ap0)
        v1.appointment = ap1
        v1.save()
    except:
        pass

    ap0.delete()
    ons.delete()
